Import Library

In [12]:
import os
import numpy as np
from PIL import Image
import numpy as np
from numpy.random import normal

Flatten

In [13]:
# Set the input directory path
input_directory = "../Dataset/Foto_Resize_50x50"

# List all folders in the input directory
folders = [f for f in os.listdir(input_directory) if os.path.isdir(os.path.join(input_directory, f))]
print(f"Found folders: {folders}")

# Initialize arrays for inputs and outputs
X = []
y = []

# Create a mapping from folder names to one-hot encoded labels
label_map = {folder: idx for idx, folder in enumerate(folders)}
num_classes = len(folders)

# Process each folder and photo
for folder in folders:
    folder_path = os.path.join(input_directory, folder)
    
    # Sort the list of photos to ensure consistent order
    photos = sorted([p for p in os.listdir(folder_path) if p.endswith(('.png', '.jpg', '.jpeg'))])
    
    for photo in photos:
        photo_path = os.path.join(folder_path, photo)
        image = Image.open(photo_path)
        
        # Convert the image to a numpy array and flatten it
        image_array = np.array(image).flatten()
        X.append(image_array)
        
        # Create a one-hot encoded label
        label = np.zeros(num_classes)
        label[label_map[folder]] = 1
        y.append(label)

# Convert lists to numpy arrays
X = np.array(X)
y = np.array(y)

print(f"Input array shape: {X.shape}")
print(f"Output array shape: {y.shape}")

Found folders: ['Azmira', 'David', 'Dimas', 'Fadhli', 'Fadlin', 'Hafidz', 'Haidar', 'Hanna', 'Keiko', 'Khansa', 'Mikhael', 'Puti', 'Raesa', 'Satwika', 'Toni']
Input array shape: (1384, 2500)
Output array shape: (1384, 15)


Normalize X, Print Y

In [14]:
print(X)
xmax = X.max()
xmin = X.min()
X =  (X-xmin)/(xmax-xmin)
print(X)

[[218 220 223 ... 149 130 111]
 [252 253 253 ... 147 132 118]
 [111 119 139 ... 222 218 218]
 ...
 [213 214 215 ...  58  50  50]
 [214 214 214 ...  75  67  67]
 [ 48  54  64 ... 215 212 212]]
[[0.85490196 0.8627451  0.8745098  ... 0.58431373 0.50980392 0.43529412]
 [0.98823529 0.99215686 0.99215686 ... 0.57647059 0.51764706 0.4627451 ]
 [0.43529412 0.46666667 0.54509804 ... 0.87058824 0.85490196 0.85490196]
 ...
 [0.83529412 0.83921569 0.84313725 ... 0.22745098 0.19607843 0.19607843]
 [0.83921569 0.83921569 0.83921569 ... 0.29411765 0.2627451  0.2627451 ]
 [0.18823529 0.21176471 0.25098039 ... 0.84313725 0.83137255 0.83137255]]


In [15]:
print(y)

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]


Initialize Layer

In [16]:
inputLayer = 2500
hiddenLayer = 7
outputLayer = 15

Calculating Weights

In [ ]:
#70x70
#input = 4900, hiddenLayer = 1, neuron = 2

n = 4900
stddev = np.sqrt(2/n)
hiddenNeurons = 2
w = np.random.normal(loc=0, scale = stddev, size = (n, hiddenNeurons))
print(w)
np.shape(w)

[[ 0.03874289 -0.03537039]
 [ 0.00995655  0.00519049]
 [ 0.032025    0.00690439]
 ...
 [ 0.01991406  0.00443412]
 [ 0.00064986  0.0417303 ]
 [-0.0120481  -0.00653353]]


(4900, 2)